# Transformt Data with the constrain of 1 seconds , breaks and encoding for NLP

In [24]:
import pickle
import numpy as np

In [25]:
# I manually move the data (locally) inside this folder 
with open('./ml4science_data_fake.pkl', 'rb') as fp:
    dl = pickle.load(fp)

In [26]:
print(dl.keys())
dl['sequences'][0].keys()

dict_keys(['sequences', 'index', 'available_demographics'])


dict_keys(['learner_id', 'sequence', 'begin', 'end', 'gender', 'year', 'language', 'label'])

In [27]:
for i in range(3):
    print(
        'begin {} - end {}: {}'.format
        (dl['sequences'][0]['begin'][i],
        dl['sequences'][0]['end'][i],
        dl['sequences'][0]['sequence'][i])
    )

begin 0 - end 7.889: [0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3440621047581669]
begin 7.889 - end 83.03: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0]
begin 83.03 - end 86.28: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14174189890531658]


## Repair FAKE data

In [28]:
for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])): 
        non_zero_count = np.count_nonzero(dl['sequences'][i]['sequence'][j])
        if non_zero_count != 2:
            #add randomly a one in of of the four first values of the sequence
            dl['sequences'][i]['sequence'][j][np.random.randint(0,4)] = 1

In [29]:
for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])): 
        non_zero_count = np.count_nonzero(dl['sequences'][i]['sequence'][j])
        if non_zero_count != 2:
            print(f"Vector at student {i}, sequence {j} has {non_zero_count} non-zero values")
            non_zero_positions = np.nonzero(dl['sequences'][i]['sequence'][j])[0]
            print(f"Positions of non-zero values: {non_zero_positions}")

# Manual check
print(dl['sequences'][20]['sequence'][1])
print(dl['sequences'][20]['sequence'][2])
print(dl['sequences'][28]['sequence'][1])
print(dl['sequences'][28]['sequence'][2])

[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3564261315115266]
[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08771929824561517, 0.0, 0.0]
[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08745293331713845]


In [30]:
# save data in a pkl file
#with open('./ml4science_data_fake_repaired.pkl', 'wb') as fp:
#    pickle.dump(dl, fp)

## Test for create Vocabulary

In [31]:
# Arguments for future function
n_states = 4
n_actions = 6
n_tokens = 3

for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])): 
        idx = np.nonzero(dl['sequences'][i]['sequence'][j])
        value = idx[0][0]*n_actions + idx[0][1] - (n_states - n_tokens -1) #Cuz want second equation to start from 1
        dl['sequences'][i]['sequence'][j] = value

In [48]:
# Breaks correspond to the 5th value of the events -> the 9th of the complete sequence
# Thus with our vocabulary it correspond to:  5 ; 11 ; 18 ; 23

for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])):

        #print((dl['sequences'][0]['end'][j] - dl['sequences'][0]['begin'][j]) > 15)
        #print((dl['sequences'][i]['sequence'][j] in [5,11,18,23]))

        if ( dl['sequences'][i]['sequence'][j] in [5,11,18,23]) and ( (dl['sequences'][0]['end'][j] - dl['sequences'][0]['begin'][j]) > 15 ): 
            
            #print((dl['sequences'][0]['end'][j] - dl['sequences'][0]['begin'][j]))
            dl['sequences'][i]['sequence'][j] = "SEP" #decide value for SEP

In [49]:
for i in range(len(dl['sequences'][0]['begin'])):
    print(
        'begin {} - end {}: {}'.format
        (dl['sequences'][0]['begin'][i],
        dl['sequences'][0]['end'][i],
        dl['sequences'][0]['sequence'][i])
    )

begin 0 - end 7.889: 24
begin 7.889 - end 83.03: SEP
begin 83.03 - end 86.28: 24
begin 86.28 - end 86.33: 19
begin 87.278 - end 90.562: 19
begin 90.562 - end 99.597: 24
begin 99.597 - end 100.32: 21
begin 101.267 - end 107.071: 23
begin 108.989 - end 114.653: 19
begin 115.424 - end 115.935: 21
begin 117.488 - end 135.421: SEP
begin 136.702 - end 144.927: 19
begin 144.927 - end 147.995: 24
begin 147.995 - end 149.956: 19
begin 150.243 - end 150.934: 19
begin 150.934 - end 153.606: 24
begin 153.606 - end 153.656: 19
begin 155.007 - end 169.149: 23
begin 170.791 - end 170.841: 1
begin 170.841 - end 173.98: 6
begin 173.98 - end 175.493: 1
begin 175.859 - end 182.973: 1
begin 184.477 - end 188.182: 1
begin 190.224 - end 195.675: 1
begin 196.178 - end 197.047: 1
begin 197.421 - end 198.127: 1
begin 198.37 - end 200.423: 1
begin 201.114 - end 203.82: 1
begin 205.115 - end 207.264: 5
begin 207.264 - end 210.048: 6
begin 210.048 - end 213.956: 2
begin 215.274 - end 231.234: SEP
begin 231.234 - 

In [ ]:
word_dict = {'[PAD]': 0, '[SEP]': 2, '[MASK]': 3}
# create a vocabulary of words based on one-hot encoding of a vector for NLP